In [4]:
import pandas as pd


# INDICATOR IMPLEMENTATIONS


In [5]:
def sma(series: pd.Series, n: int):
    """Simple Moving Average"""
    return series.rolling(int(n)).mean()

In [6]:
def rsi(series: pd.Series, n: int):
    """Relative Strength Index"""
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    
    avg_gain = gain.rolling(n).mean()
    avg_loss = loss.rolling(n).mean()
    
    rs = avg_gain / (avg_loss.replace(0, 1e-9))
    return 100 - (100 / (1 + rs))

# CODE GENERATION HELPERS

In [7]:

def _resolve_operand(node, df_name="df"):
    """Convert AST operand nodes → Python code strings."""
    
    # A series like close, volume
    if node["type"] == "series":
        val = node["value"]
        if val == "yesterday_high":
            return f"{df_name}['high'].shift(1)"   # special handling
        return f"{df_name}['{val}']"
    
    # An indicator like sma(close,20)
    if node["type"] == "indicator":
        name = node["name"]
        series = node["params"][0]
        period = node["params"][1]
        
        if name == "sma":
            return f"sma({df_name}['{series}'], {period})"
        if name == "rsi":
            return f"rsi({df_name}['{series}'], {period})"
    
    return str(node)  # fallback


In [8]:


def _expr_from_ast(node, df_name="df"):
    """Turn an AST node into executable Python expression."""
    
    t = node["type"]
    
    # Binary comparison like >, <, >=
    if t == "binary_op":
        left = _resolve_operand(node["left"], df_name)
        right = _resolve_operand(node["right"], df_name)
        op = node["op"]
        return f"({left} {op} {right})"
    
    # Cross events
    if t == "cross":
        left = _resolve_operand(node["left"], df_name)
        right = _resolve_operand(node["right"], df_name)
        
        if node["dir"] == "above":
            return f"(({left} > {right}) & ({left}.shift(1) <= {right}.shift(1)))"
        
        if node["dir"] == "below":
            return f"(({left} < {right}) & ({left}.shift(1) >= {right}.shift(1)))"
    
    # Boolean AND
    if t == "and":
        left = _expr_from_ast(node["left"], df_name)
        right = _expr_from_ast(node["right"], df_name)
        return f"({left} & {right})"
    
    # Boolean OR
    if t == "or":
        left = _expr_from_ast(node["left"], df_name)
        right = _expr_from_ast(node["right"], df_name)
        return f"({left} | {right})"
    
    raise ValueError(f"Unknown AST node type: {t}")


# MAIN FUNCTION — CODE GENERATOR


In [9]:


def generate_strategy_code(ast: dict, df_name="df"):
    """
    Takes AST dictionary and returns a Python function definition
    that evaluates entry/exit signals on a pandas DataFrame.
    """
    
    entry_parts = []
    exit_parts = []
    
    for node in ast.get("entry", []):
        entry_parts.append(_expr_from_ast(node, df_name))
    
    for node in ast.get("exit", []):
        exit_parts.append(_expr_from_ast(node, df_name))
    
    # Combine with AND/OR inside DSL
    entry_expr = " & ".join(entry_parts) if entry_parts else "pd.Series(False, index=df.index)"
    exit_expr = " | ".join(exit_parts) if exit_parts else "pd.Series(False, index=df.index)"
    
    func_code = f"""
def evaluate_signals({df_name}):
    signals = pd.DataFrame(index={df_name}.index)
    signals['entry'] = {entry_expr}
    signals['exit'] = {exit_expr}
    return signals
"""
    
    return func_code
